<a href="https://colab.research.google.com/github/arinaaandreeva/ML_lab_CV/blob/main/Lab_CV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import cv2
import numpy as np
import os
import pandas as pd
import torch
import torch.nn.functional as F
from torchvision import transforms

In [2]:
# Вводим данные из json, который получаем на kaggle

! pip install -q kaggle
from google.colab import files
files.upload()
! mkdir ~/.kaggle
import kagglehub
kagglehub.login()
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [3]:
"""Загрузка файлов с Kaggle"""
!kaggle competitions download -c cv-lab-dm-24
#!unzip cv-lab-dm-24.zip -d data

 99% 549M/553M [00:07<00:00, 84.7MB/s]
100% 553M/553M [00:07<00:00, 82.5MB/s]


In [4]:
import zipfile
with zipfile.ZipFile('/content/cv-lab-dm-24.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/data')

In [ ]:
"""
    Файлы COCO
    Их позже подгружают в модель (например, чере torchvision.datasets.CocoDetection)
    В json содржится информация о bounding box, area & category_id
    Категория (category_id) используется как таргет
    """
import json
with open('/content/data/usdc_train.json', 'r') as f:
    data = json.load(f)

with open('/content/data/test_file_names.json', 'r') as f:
    data_test = json.load(f)

# # Извлечение категорий и изображений
# categories = data['categories']  # Список категорий
# images = data['images']  # Список изображений
# annotations = data['annotations']  # Список аннотаций к изображениям

# # Преобразование в DataFrame (если необходимо)
# categories_df = pd.DataFrame(categories)
# images_df = pd.DataFrame(images)
# annotations_df = pd.DataFrame(annotations)


## Задание 1.
### Данные без нормализации

In [ ]:
"""Загрузка данных без обработки
    Нужно раскомментировать код, чтобы загрузилось"""
def load_images(folder_path):
    images = []
    for filename in os.listdir(folder_path):
        if filename.endswith('.jpg'): # у нас файлы jpg
            filepath = os.path.join(folder_path, filename)
            img = cv2.imread(filepath) # пиксели имеют тип uint8 (целое число от 0 до 255)
            if img is not None:
                images.append((filename, img))
    return images


# Загружаем изображения из папок
train_path = "/content/data/train_images/train_images"
test_path = '/content/data/test_images/test_images'
#train_images = load_images(train_path)
#test_images = load_images(test_path)

###Local Contrast Normalization (LCN)

In [ ]:
def local_contrast_normalization(img, kernel_size=8):
    """
    Выполняет Local Contrast Normalization (LCN).
    kernel_size: Размер локального окна (например, 8x8).
    """
    # Преобразуем изображение в float32
    img = img.astype(np.float32) / 255.0

    # Вычисляем локальное среднее значение
    mean = cv2.blur(img, (kernel_size, kernel_size))

    # Вычисляем локальное стандартное отклонение
    squared_diff = cv2.blur((img - mean) ** 2, (kernel_size, kernel_size))
    std = np.sqrt(squared_diff+ 1e-5) # добавляем епсилон, чтобы не было деления на 0

    # Применяем нормализацию
    normalized_img = (img - mean) / std

    return normalized_img

"""
    Применяет LCN ко всем изображениях в указанной папке.
    Сохраняет обработанные изображения в указанную выходную папку.
    """

def apply_lcn_to_images(folder_path, output_folder):

    # Проверяем, существует ли выходная папка, если нет — создаем её
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Обрабатываем все изображения в папке
    for filename in os.listdir(folder_path):
        if filename.endswith('.jpg'):  # Обрабатываем только файлы .jpg
            filepath = os.path.join(folder_path, filename)
            img = cv2.imread(filepath)  # Загружаем изображение (тип uint8)

            if img is not None:
                # Преобразуем в grayscale (если изображение цветное)
                if len(img.shape) == 3:
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

                # Применяем Local Contrast Normalization
                normalized_img = local_contrast_normalization(img)

                # Преобразуем изображение обратно в uint8 для сохранения
                normalized_img_uint8 = np.clip(normalized_img * 255, 0, 255).astype(np.uint8)

                # Сохраняем результат в выходную папку с оригинальным именем
                output_path = os.path.join(output_folder, filename)
                cv2.imwrite(output_path, normalized_img_uint8)

output_folder = '/content/data/lcn_images'  # Папка для сохранения обработанных изображений

apply_lcn_to_images(train_path, output_folder)

In [ ]:
"""Через torch, x- изображение"""
# def local_contrast_normalization(x, kernel_size=7):
#     x = x.float()
#     # Локальное среднее
#     mean = F.avg_pool2d(x, kernel_size, stride=1, padding=kernel_size//2)
#     # Локальное стандартное отклонение
#     squared_diff = F.avg_pool2d((x - mean) ** 2, kernel_size, stride=1, padding=kernel_size//2)
#     std = torch.sqrt(squared_diff + 1e-5)  # Добавляем небольшой эпсилон для устойчивости

#     return (x - mean) / std


'Через torch, x- изображение'

### Local response normalization

In [ ]:
def local_response_normalization(x, radius=5, alpha=1.0, beta=0.75, k=2.0):
    """
    Применяет Local Response Normalization (LRN) к тензору x.

    :param x: Входной тензор (например, изображение).
    :param radius: Радиус локальной нормализации.
    :param alpha: Коэффициент масштабирования.
    :param beta: Экспоненциальный коэффициент.
    :param k: Смещение для улучшения устойчивости.
    :return: Нормализованный тензор.
    """
    x = x.float()

    # Размер ядра для свертки (с учетом радиуса)
    kernel_size = 2 * radius + 1

    # Создание весов для свертки
    weights = torch.ones(x.size(1), 1, kernel_size, kernel_size).to(x.device)  # Один вес для каждого канала

    # Процесс локальной нормализации (свёртка с квадратами значений)
    square_sum = F.conv2d(x**2, weight=weights, stride=1, padding=radius, groups=x.size(1))

    # Нормализация
    norm = (alpha * square_sum + k) ** beta
    return x / norm

In [ ]:
def normalization_lrn(folder_path, output_folder, radius=5, alpha=1.0, beta=0.75, k=2.0):
    """
    Обрабатывает все изображения в папке и применяет Local Response Normalization (LRN),
    затем сохраняет результат в другую папку.

    :param folder_path: Путь к папке с изображениями.
    :param output_folder: Путь к папке для сохранения обработанных изображений.
    :param radius: Радиус для LRN.
    :param alpha: Коэффициент масштабирования.
    :param beta: Экспоненциальный коэффициент.
    :param k: Смещение для LRN.
    """
    # Создаем папку для вывода, если она не существует
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Трансформатор для преобразования изображения в тензор
    transform = transforms.ToTensor()

    # Обрабатываем все изображения в папке
    for filename in os.listdir(folder_path):
        if filename.endswith('.jpg') or filename.endswith('.png'):  # Убедитесь, что работает с изображениями
            filepath = os.path.join(folder_path, filename)

            # Загружаем изображение
            img = Image.open(filepath).convert('RGB')  # Преобразуем в RGB, если изображение не в этом формате

            # Преобразуем изображение в тензор
            x = transform(img).unsqueeze(0)  # Добавляем размерность для batch_size, т.е. (1, 3, H, W)

            # Применяем Local Response Normalization (LRN)
            normalized_x = local_response_normalization(x, radius, alpha, beta, k)

            # Преобразуем тензор обратно в изображение и сохраняем
            normalized_img = normalized_x.squeeze(0).clamp(0, 1).numpy().transpose(1, 2, 0)  # (H, W, C)
            normalized_img = (normalized_img * 255).astype('uint8')  # Преобразуем в диапазон 0-255 для сохранения

            # Сохраняем обработанное изображение
            output_path = os.path.join(output_folder, filename)
            cv2.imwrite(output_path, cv2.cvtColor(normalized_img, cv2.COLOR_RGB2BGR))  # Сохраняем в BGR

    print(f"Обработка завершена. Изображения сохранены в {output_folder}")

# output_folder = '/content/data/lrn_images'  # Путь к папке для сохранения нормализованных изображений

# normalization_lrn(train_path, output_folder, radius=5, alpha=1.0, beta=0.75, k=2.0)

В современных задачах обнаружения объектов на автопилоте LRN не является оптимальным выбором, так как он устарел.когда объекты плохо видны из-за изменений контраста или освещенности, Local Contrast Normalization (LCN) может быть полезным методом. Это может помочь сделать детали объектов более явными и улучшить их обнаружение в условиях темноты, тумана и т.п.

In [ ]:
# поэтому используем LCN и достаем из папки данные
#train_images = load_images('/content/data/lcn_images')

### Transform

In [ ]:
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

In [ ]:
transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.ColorJitter(brightness=0.5, hue=0.3),
    A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.2, rotate_limit=45, p=0.2),#affine
    #ToTensorV2(),
], bbox_params=A.BboxParams(format='coco'))

In [ ]:
WORK_DIR = '/content/data'

In [ ]:
for i in range(len(data['annotations'])):
  for j in range(2):
    if(data['annotations'][i]['bbox'][j] + data['annotations'][i]['bbox'][j+2] >= 512):
      data['annotations'][i]['bbox'][j+2] -= 3
    if(data['annotations'][i]['bbox'][j] < 0):
      data['annotations'][i]['bbox'][j] = 0

import os
import json
import cv2
import numpy as np
import albumentations as A
from albumentations.pytorch import ToTensorV2
from tqdm import tqdm

transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.ColorJitter(brightness=0.5, hue=0.3),
    A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.2, rotate_limit=45, p=0.2),#affine
    A.Resize(224, 224),
    #ToTensorV2(),
], bbox_params=A.BboxParams(format='coco', label_fields=['class_labels']))


transformed = []
for annotation in tqdm(data['images']):

    image_id = annotation['id']
    image_path = os.path.join(WORK_DIR + '/lcn_images', annotation['file_name'])

    image = cv2.imread(image_path)

    bboxes = []
    class_labels = []
    for ann in data['annotations']:
        if ann['image_id'] == image_id:
            bbox = ann['bbox']
            bboxes.append(bbox)
            class_labels.append(ann['category_id'])

    transformed.append(transform(image=image, bboxes=np.array(bboxes), class_labels=class_labels))


100%|██████████| 12000/12000 [07:07<00:00, 28.08it/s]


# Модели

In [ ]:
import os
import json
import cv2
import numpy as np
from tqdm import tqdm
from PIL import Image

import albumentations as A
from albumentations.pytorch import ToTensorV2
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
def local_contrast_normalization(img, kernel_size=8, epsilon=1e-5):
    """
    Выполняет Local Contrast Normalization (LCN).
    kernel_size: Размер локального окна (например, 8x8).
    """
    # Преобразуем изображение в float32
    img = img.astype(np.float32) / 255.0

    # Вычисляем локальное среднее значение
    mean = cv2.blur(img, (kernel_size, kernel_size))

    # Вычисляем локальное стандартное отклонение
    squared_diff = cv2.blur((img - mean) ** 2, (kernel_size, kernel_size))
    std = np.sqrt(squared_diff+ epsilon) # добавляем епсилон, чтобы не было деления на 0

    # Применяем нормализацию
    normalized_img = (img - mean) / std

    return normalized_img

### Create your own model and train it
Начали с малого, модель, которая предсказывает только category_id

In [ ]:
def get_image_annotations(data, image_id):
    """
    Extract class labels for a given image ID from COCO annotations.
    """
    class_labels = []
    for annotation in data['annotations']:
        if annotation['image_id'] == image_id:
            class_labels.append(annotation['category_id'])  # Use category_id as the target
    return class_labels



def collate_fn(batch):
    images = []
    labels = []

    for image, label in batch:
      if label != -1:
        images.append(image)
        labels.append(label)

    # Преобразуем изображения в тензор
    images = torch.stack(images, dim=0)
    labels = torch.tensor(labels, dtype=torch.long)

    return images, labels


class CustomDataset(Dataset):
    def __init__(self, data, image_dir, transforms=None, apply_lcn=False):
        self.data = data
        self.image_dir = image_dir
        self.transforms = transforms
        self.apply_lcn = apply_lcn

    def __len__(self):
        return len(self.data['images'])

    def __getitem__(self, idx):
        image_info = self.data['images'][idx]
        image_id = image_info['id']
        image_path = os.path.join(self.image_dir, image_info['file_name'])

        image = cv2.imread(image_path)
        if image is None:
            raise ValueError(f"Failed to load image: {image_path}")
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        if self.apply_lcn:
            image = local_contrast_normalization(image)

        # Получаем метки классов
        class_labels = get_image_annotations(self.data, image_id)

        # Для классификации используем только один объект (например, первый)
        label = class_labels[0] if class_labels else 12  # -1 для случаев без аннотации

        if self.transforms:
            transformed = self.transforms(image=image)
            image = transformed['image']

        return image, label




# Load data from JSON
with open('/content/data/usdc_train.json') as f:
    train_data = json.load(f)

with open('/content/data/test_file_names.json') as f:
    test_data = json.load(f)

train_transforms = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.ColorJitter(brightness=0.5, hue=0.3),
    A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.2, rotate_limit=45, p=0.2),
    A.Resize(128, 128),
    ToTensorV2()
])

test_transforms = A.Compose([
    A.Resize(224, 224),
    ToTensorV2()
])


# Create datasets and dataloaders
train_dataset = CustomDataset(train_data, "/content/data/train_images/train_images", transforms=train_transforms, apply_lcn=True)
test_dataset = CustomDataset(test_data, "/content/data/test_images/test_images", transforms=test_transforms, apply_lcn=True)

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True,collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False,collate_fn=collate_fn)


In [ ]:
import torch
import torch.nn as nn

import torch
import torch.nn as nn

class CustomClassifier(nn.Module):
    def __init__(self, num_classes):
        super(CustomClassifier, self).__init__()
        # Сверточная часть (backbone)
        self.backbone = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),  # Уменьшает размер в 2 раза
            nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),  # Ещё уменьшение размера
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)  # Ещё уменьшение
        )

        # Классификационная голова
        self.global_pool = nn.AdaptiveAvgPool2d((1, 1))  # Сводим к фиксированному размеру
        self.classifier = nn.Linear(64, num_classes)  # Выход: количество классов

    def forward(self, x):
        features = self.backbone(x)
        pooled_features = self.global_pool(features).flatten(1)
        logits = self.classifier(pooled_features)
        return logits



In [ ]:
import torch.optim as optim
import torch.nn.functional as F
from tqdm import tqdm


num_classes = 13  # Укажите количество классов
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CustomClassifier(num_classes=num_classes).to(device)

# Оптимизатор и функция потерь
optimizer = optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss()

# Обучение
def train_model(model, train_loader, optimizer, num_epochs=10):
    model.train()
    for epoch in tqdm(range(num_epochs)):
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

train_model(model, train_loader, optimizer, num_epochs=5)



 20%|██        | 1/5 [09:45<39:03, 585.99s/it]

Epoch [1/5], Loss: 1.1939


 40%|████      | 2/5 [19:37<29:27, 589.11s/it]

Epoch [2/5], Loss: 1.1541


 60%|██████    | 3/5 [29:23<19:35, 587.84s/it]

Epoch [3/5], Loss: 1.1469


 80%|████████  | 4/5 [39:11<09:47, 587.98s/it]

Epoch [4/5], Loss: 1.1413


100%|██████████| 5/5 [48:58<00:00, 587.64s/it]

Epoch [5/5], Loss: 1.1344


In [ ]:
import joblib
# save
joblib.dump(model, "model_cv1.pkl")

['model.pkl']

In [ ]:
model.predict(test_loader)

### Модель, которая выводит уже и bbox и class, но попытки неуспешны

In [ ]:
import os
import json
import cv2
import numpy as np
from torch.utils.data import Dataset
import albumentations as A
from albumentations.pytorch import ToTensorV2
import torch
import torch.nn as nn
from torchvision import models
class ObjectDetectionDataset(Dataset):
    def __init__(self, annotations_file, image_dir, transforms=None):
        with open(annotations_file, 'r') as f:
            self.data = json.load(f)  # загружаем аннотации
        self.image_dir = image_dir
        self.transforms = transforms

    def __len__(self):
        return len(self.data['images'])

    def __getitem__(self, idx):
      image_info = self.data['images'][idx]
      image_id = image_info['id']
      image_path = os.path.join(self.image_dir, image_info['file_name'])

      # Загружаем изображение
      image = cv2.imread(image_path)
      image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

      # Получаем аннотации для изображения
      annotations = [anno for anno in self.data['annotations'] if anno['image_id'] == image_id]

      bboxes = []
      labels = []
      for anno in annotations:
          bboxes.append(anno['bbox'])  # формат [x, y, w, h]
          labels.append(anno['category_id'])

      # Преобразуем в numpy массивы
      bboxes = np.array(bboxes) if bboxes else np.zeros((0, 4))
      labels = np.array(labels) if labels else np.zeros((13, 4))

      # Применяем трансформации, если они заданы
      if self.transforms:
          augmented = self.transforms(image=image, bboxes=bboxes, class_labels=labels)
          image = augmented['image']
          bboxes = augmented['bboxes']
          labels = augmented['class_labels']

      return image, bboxes, labels

def custom_collate_fn(batch):
    images = []
    bboxes = []
    labels = []

    for image, bbox, label in batch:
        images.append(image)  # Add image tensor
        bboxes.append(torch.tensor(bbox, dtype=torch.float32))  # Add bbox tensor
        labels.append(torch.tensor(label, dtype=torch.long))  # Add label tensor

    # Stack images into a single tensor [batch_size, C, H, W]
    images = torch.stack(images, dim=0)

    return images, bboxes, labels

train_transforms = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.ColorJitter(brightness=0.5, hue=0.3),
    A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.2, rotate_limit=45, p=0.2),
    A.Resize(128, 128),
    ToTensorV2()
], bbox_params=A.BboxParams(format='coco', label_fields=['class_labels']))

# train_transforms = A.Compose([
#     A.Resize(224, 224),
#     A.HorizontalFlip(p=0.5),
#     A.ColorJitter(brightness=0.5, hue=0.3),
#     ToTensorV2()  # Ensures the image is converted to a tensor
# ]


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models
import torch
import torch.nn as nn
from torchvision import models

class ObjectDetectionModel(nn.Module):
    def __init__(self, num_classes):
        super(ObjectDetectionModel, self).__init__()

        # Используем предобученный ResNet для извлечения признаков (backbone)
        resnet = models.resnet18(pretrained=True)
        # Убираем последний слой классификации (FC) и avgpool
        self.backbone = nn.Sequential(*list(resnet.children())[:-2])  # Убираем avgpool и fc

        # Добавляем адаптивный слой пулинга для фиксированного выхода
        self.global_pool = nn.AdaptiveAvgPool2d((1, 1))  # Используем глобальный пулинг

        # Пример фиктивного входа для определения размерности признаков
        dummy_input = torch.zeros(1, 3, 224, 224)  # Имитация входа размером (batch_size, 3, 224, 224)
        with torch.no_grad():
            dummy_features = self.backbone(dummy_input)  # Получаем фичи после backbone
            dummy_features = self.global_pool(dummy_features)  # Применяем глобальный пулинг
            dummy_features = dummy_features.view(dummy_features.size(0), -1)  # Преобразуем в (batch_size, feature_dim)
            feature_dim = dummy_features.size(1)  # Получаем размерность фичей

        # Классификация (для категорий объектов)
        self.classifier = nn.Linear(feature_dim, num_classes)

        # Регрессия (для bbox)
        self.bbox_regressor = nn.Linear(feature_dim, 4)  # 4 координаты для bbox [x, y, w, h]

        # Оценка уверенности (objectness score)
        self.objectness = nn.Linear(feature_dim, 1)  # для objectness

    def forward(self, x):
        # Извлечение признаков через backbone
        features = self.backbone(x)  # (batch_size, channels, h, w)
        features = self.global_pool(features)  # (batch_size, channels, 1, 1)
        features = features.view(features.size(0), -1)  # Преобразуем в (batch_size, channels)

        # Прогнозы
        class_preds = self.classifier(features)  # Прогноз категории
        bbox_preds = self.bbox_regressor(features)  # Прогноз bbox
        score_preds = torch.sigmoid(self.objectness(features))  # Прогноз уверенности (sigmoid для значения от 0 до 1)

        return class_preds, bbox_preds, score_preds


In [ ]:
import torch
from tqdm import tqdm
from torch.utils.data import DataLoader

# Настроим устройство
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Инициализация модели
num_classes = 13  # Общее количество классов (включая фон)
model = ObjectDetectionModel(num_classes=num_classes).to(device)

# Определим функции потерь
criterion_class = nn.CrossEntropyLoss()  # Для классификации
criterion_bbox = nn.SmoothL1Loss()  # Для регрессии bbox
criterion_obj = nn.BCELoss()  # Для objectness score

# Оптимизатор
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

# Датасет и DataLoader
train_dataset = ObjectDetectionDataset(
    annotations_file='/content/data/usdc_train.json',
    image_dir= '/content/data/train_images/train_images', #'/content/data/lcn_images',
    transforms=train_transforms
)


train_loader = DataLoader(
    train_dataset,
    batch_size=4,
    shuffle=True,
    collate_fn=custom_collate_fn
)


def train_model(model, train_loader, optimizer, num_epochs):
    model.train()
    for epoch in tqdm(range(num_epochs), desc="Training Epochs"):
        running_loss = 0.0
        for images, bboxes, labels in train_loader:
            images = images.to(device)

            # Forward pass
            optimizer.zero_grad()
            class_preds, bbox_preds, score_preds = model(images/255)

            # Calculate losses
            loss_class = 0.0
            loss_bbox = 0.0

            for i in range(len(bboxes)):
                target_bboxes = bboxes[i].to(device)
                target_labels = labels[i].to(device)
                print(class_preds[i], bbox_preds[i], target_labels)

                loss_class += criterion_class(class_preds[i], target_labels)
                loss_bbox += criterion_bbox(bbox_preds[i], target_bboxes)

            # Normalize the loss by batch size
            loss_class /= len(bboxes)
            loss_bbox /= len(bboxes)
            loss = loss_class + loss_bbox

            # Backward pass
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")


# Запуск обучения
train_model(model, train_loader, optimizer, num_epochs=5)


Training Epochs:   0%|          | 0/5 [00:00<?, ?it/s]

tensor([ 0.4885,  0.2034,  0.9696,  0.7145,  0.8864, -0.2270, -0.2932, -1.4934,
        -2.1856, -1.9779, -1.4708, -0.6105, -0.0426],
       grad_fn=<SelectBackward0>) tensor([ 1.3415, -0.0061, -0.1427,  0.8949], grad_fn=<SelectBackward0>) tensor([], dtype=torch.int64)


RuntimeError: size mismatch (got input: [13], target: [0])

In [ ]:
def predict(model, image):
    model.eval()
    image = image.unsqueeze(0).to(device)  # Добавляем размерность для батча
    with torch.no_grad():
        class_preds, bbox_preds, score_preds = model(image)

        # Получаем классы, bbox и уверенность
        predicted_class = torch.argmax(class_preds, dim=1)
        predicted_bbox = bbox_preds.squeeze(0)  # Для одного изображения
        predicted_score = torch.sigmoid(score_preds).squeeze(0)  # Для уверенности

        return predicted_class, predicted_bbox, predicted_score
